# Loading Model 

In [13]:
import os
import sys
from tensorflow.keras.optimizers import Adadelta
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape, Activation
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv1D, Lambda
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [14]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [15]:
files_location = '/content/gdrive/Colab_Notebooks/temp_files/'
sys.path.append(os.path.abspath(files_location))

In [16]:
con_win_size = 9
num_strings = 6
num_classes = 21
input_shape = (192, con_win_size, 1)

def softmax_by_string(t):
    sh = K.shape(t)
    string_sm = []
    for i in range(num_strings):
        string_sm.append(K.expand_dims(K.softmax(t[:, i, :]), axis=1))
    return K.concatenate(string_sm, axis=1)

def build_model():

    model = Sequential()
    model.add(
        Conv2D(32,
               kernel_size=(3, 3),
               activation="relu",
               input_shape=input_shape))
    model.add(Conv2D(64, (3, 3), activation="relu"))
    model.add(Conv2D(64, (3, 3), activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes * num_strings))  # no activation
    model.add(Reshape((num_strings, num_classes)))
    model.add(Activation(softmax_by_string))

#     model.compile(loss=self.catcross_by_string,
#                   optimizer=Adadelta(),
#                   metrics=[self.avg_acc])

    return model
model = build_model()

# Load model weights 

In [17]:
!ls

00_BN1-129-Eb_comp_mic.wav
00_BN1-129-Eb_comp.npz
00_Jazz3-137-Eb_solo_full_val0_75acc_predictions.npz
DataGenerator.py
experimentmono.wav
experiment.wav
full_val0_75acc_weights.h5
Metrics.py
__pycache__
TabCNN.py
weights_02_loss_6.2780_acc_0.7276.h5


In [18]:
%cd gdrive/MyDrive/Colab_Notebooks/temp_files
%pwd

[Errno 2] No such file or directory: 'gdrive/MyDrive/Colab_Notebooks/temp_files'
/content/gdrive/MyDrive/Colab_Notebooks/temp_files


'/content/gdrive/My Drive/Colab_Notebooks/temp_files'

In [19]:
!ls

00_BN1-129-Eb_comp_mic.wav
00_BN1-129-Eb_comp.npz
00_Jazz3-137-Eb_solo_full_val0_75acc_predictions.npz
DataGenerator.py
experimentmono.wav
experiment.wav
full_val0_75acc_weights.h5
Metrics.py
__pycache__
TabCNN.py
weights_02_loss_6.2780_acc_0.7276.h5


In [20]:
model.load_weights('full_val0_75acc_weights.h5')

In [21]:
model

# Load from wav file


In [22]:
#!pip install jams

In [23]:
import os
import numpy as np
import jams
from scipy.io import wavfile
import sys
import librosa
from tensorflow.keras.utils import to_categorical


class TabDataReprGen:
    def __init__(self, mode="c"):
        # file path to the GuitarSet dataset
        path = "data/GuitarSet/"
        self.path_audio = path + "audio/audio_mic/"
        self.path_anno = path + "annotation/"

        # labeling parameters
        self.string_midi_pitches = [40, 45, 50, 55, 59, 64]
        self.highest_fret = 19
        self.num_classes = self.highest_fret + 2  # for open/closed

        # prepresentation and its labels storage
        self.output = {}

        # preprocessing modes
        #
        # c = cqt
        # m = melspec
        # cm = cqt + melspec
        # s = stft
        #
        self.preproc_mode = mode    # Preprocessing mode for the wav file data
        self.downsample = True      # Select to lower sample rate of data
        self.normalize = True       # Select to normalize data
        self.sr_downs = 22050       # Lowered sample rate

        # CQT parameters
        self.cqt_n_bins = 192           # Number of bins for the constant-Q transform "c"
        self.cqt_bins_per_octave = 24   # Number of bins per octave

        # STFT parameters
        self.n_fft = 2048       # Length of the FFT window
        self.hop_length = 512   # Number of samples between successive frames

        # save file path
        self.save_path = "data/spec_repr/" + self.preproc_mode + "/"

    
    def load_rep_from_raw_file(self, filename):
        file_audio = filename
        self.sr_original, data = wavfile.read(file_audio)
        self.sr_curr = self.sr_original
        # preprocess audio, store in output dict
        self.output["repr"] = np.swapaxes(self.preprocess_audio(data), 0, 1)
        return self.output["repr"]

    def preprocess_audio(self, data):
        """
        Preprocesses data depending on mode selected using librosa.
        It converts data to float, then it normalizes it and resamples it
        to a lower sample rate. Then, preprocesses it and returns the processed data
            Args:
                data ([np.array]): [data created by wavfile.read]
            Returns:
                [np.ndarrray[shape=(n_bins, t)]]: [preprocessed data array]
        """
        data = data.astype(float)
        if self.normalize:
            data = librosa.util.normalize(data)
        if self.downsample:
            data = librosa.resample(data, self.sr_original, self.sr_downs)
            self.sr_curr = self.sr_downs
        if self.preproc_mode == "c":
            data = np.abs(
                librosa.cqt(data,     # Computes the constant-Q transform of an audio signal
                            hop_length=self.hop_length,
                            sr=self.sr_curr,        # data sample rate
                            n_bins=self.cqt_n_bins,
                            bins_per_octave=self.cqt_bins_per_octave))
        else:
            print("invalid representation mode.")

        return data


In [24]:
def repr_to_xnew(data_repr):
  halfwin = con_win_size // 2
  # full x is the entire song padded with halfwin*2 frames
  full_x = np.pad(data_repr, [(halfwin, halfwin), (0, 0)],mode='constant')
  x_new = []
  for frame_idx in range(0, len(data_repr)):  # for all frames in the experiment
      sample_x = full_x[frame_idx:frame_idx + con_win_size]
      x_new.append(np.expand_dims(np.swapaxes(sample_x, 0, 1), -1))
  x_new = np.array(x_new, dtype='float32')
  return x_new

## Importing wav file

In [25]:
#filenames = '../raw_data/experimentmono.wav'
#filenames="00_Funk3-112-C#_solo_mic.wav"
filenames = '00_BN1-129-Eb_comp_mic.wav'
genrep = TabDataReprGen()
process = genrep.load_rep_from_raw_file(filenames)

process.shape

(962, 192)

In [26]:
x_new = repr_to_xnew(process)
x_new.shape

(962, 192, 9, 1)

### Predicting from model

In [27]:
y_pred = model.predict(x_new)
y_pred.shape

(962, 6, 21)

### Tab functions


In [28]:
def make_empty_tab():
    tab_dict = {
    'E': [],
    'A': [],
    'D': [],
    'G': [],
    'B': [],
    'e': [],
    }

    tablature = pd.DataFrame.from_dict(tab_dict)
    tablature = tablature.T
    return tablature

In [29]:
def make_tab(labels, num_frames=99):
    tablature = make_empty_tab()
    # loop over all frames to add for all frame (lets test for num_frames frames)
    for frame_idx in range(0, num_frames):
        frame = pd.DataFrame(
            labels[frame_idx])  # loading frame frame_idx into a dataframe
        # now set (Frame, 0 Fret, where value < 0.9) set as 0
        # frame[0].replace(to_replace=list(frame[0][frame[0] < 0.9]),
        #                  value=0,
        #                  inplace=True)
        #now use idmax to check best fret for every string
        fret = list(frame.idxmax(axis='columns'))
        #add fret to tablature
        tablature[frame_idx] = fret
    return tablature - 1  #decreasing by 1 so that 0 fret becomes -1 and 1 fret becomes 0

In [30]:
def make_squeezed_tab(tablature, n=5):
    """Make displayable squeezed tablature
        Args:
        tablature : the full tablature made from labels
        n : the frame squeeze window. Currently using same size as con-win
    """
    #now do it for all n frames
    squeezed_Tab = make_empty_tab()
    for batch_idx in range(0, tablature.shape[1], n):
        # for every frame_batch
        frame_batch = tablature.loc[:, batch_idx:batch_idx + n - 1]
        squeezed_Tab[batch_idx] = frame_batch.mode(axis='columns')[0]
    return squeezed_Tab.astype(int)

In [31]:
def str_row(row):
  """
  Takes a row of the dataframe and returns it as a processed string
  """
  row_list=row.values.tolist()[0]
  row_list = [str(item) for item in row_list]
  joined = ''.join(row_list)
  joined = joined.replace("-1", "-")      # Replaces -1 with -
  joined = '|'.join([joined[i:i+16] for i in range(0, len(joined), 16)])
  return(joined)                          # Returns long string

def print_tabs(tabs):
  """
  Takes a tabs dataframe and prints it as a formatted tablature
  """
  tab_list = ['e','B','G','D','A','E']
  len_long_row = len(str_row(tabs.loc[['e']]))
  num_div = 4                                  # Number of divisions per line
  num_lines = int(len_long_row/17//num_div+1)  # Number of lines for the tab
  for line in range(0,num_lines):              # Iterates over lines
    for index in tab_list:                     # Iterates over each index
      row = tabs.loc[[f'{index}']]             
      long_row = str_row(row)
      print(f"{index}|{long_row[num_div*17*line:num_div*17*(line+1)]}")
    print()

In [32]:
def web_tabs(tabs):
  """
  Takes a tabs dataframe and returns a multilinear string to feed the app
  """
  tab_list = ['e','B','G','D','A','E']
  len_long_row = len(str_row(tabs.loc[['e']]))
  num_div = 4                                  # Number of divisions per line
  num_lines = int(len_long_row/17//num_div+1)  # Number of lines for the tab
  line_list = []
  for line in range(0,num_lines):              # Iterates over lines
    for index in tab_list:                     # Iterates over each index
      row = tabs.loc[[f'{index}']]             
      long_row = str_row(row)
      line_list.append(f"{index}|{long_row[num_div*17*line:num_div*17*(line+1)]}")
    line_list.append("\n")
  multi_str = '\n'.join([line for line in line_list]) # joins all lines into a multistring
  return multi_str

## Building tabs for wav file

In [33]:
labels = y_pred
print(labels[0].shape)
pd.DataFrame(labels[0])

(6, 21)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,0.988230,0.002210,0.000626,0.001942,0.002545,0.001459,0.000886,0.000823,0.000488,0.000211,0.000122,0.000047,0.000109,0.000042,0.000039,0.000027,0.000030,0.000041,0.000043,0.000033,0.000048
1,0.960075,0.001720,0.002194,0.002852,0.004566,0.006323,0.003879,0.011995,0.002296,0.001580,0.001220,0.000468,0.000289,0.000142,0.000081,0.000118,0.000045,0.000039,0.000049,0.000042,0.000028
2,0.964395,0.001366,0.000462,0.001902,0.003949,0.002878,0.002453,0.007557,0.002328,0.007809,0.001501,0.001856,0.000746,0.000242,0.000219,0.000091,0.000069,0.000034,0.000029,0.000076,0.000038
3,0.960501,0.001347,0.002054,0.003104,0.002158,0.003872,0.002356,0.007342,0.005369,0.003205,0.003539,0.002386,0.001183,0.000500,0.000627,0.000092,0.000108,0.000043,0.000100,0.000058,0.000057
4,0.955560,0.001906,0.001935,0.002801,0.001676,0.003710,0.003595,0.008279,0.003369,0.005775,0.004094,0.003018,0.001433,0.001044,0.001020,0.000144,0.000313,0.000156,0.000079,0.000041,0.000053
5,0.982484,0.001732,0.000632,0.001600,0.001520,0.001213,0.001044,0.002396,0.001519,0.001828,0.000840,0.001249,0.000651,0.000530,0.000456,0.000077,0.000027,0.000106,0.000037,0.000026,0.000032


In [34]:
#Building squeezed tabs
expanded_tab = make_tab(labels,len(labels))
display_tab = make_squeezed_tab(expanded_tab,5)
print_tabs(display_tab)

e|----------------|----------------|----------------|----------------|
B|-----------88---|-------8888-----|---------------8|8---------------|
G|-----------7773-|----------------|----------------|-------7--------|
D|--------------88|888--------88---|---------8----5-|-----------88-6-|
A|----------------|----6-----------|---------------6|6---------------|
E|----------------|----------------|----------------|----------------|

e|----------------|----------------|----------------|----------------|
B|-------66------6|----------------|---------------8|----------------|
G|----------------|----------------|----------------|----------------|
D|6666--66------66|--------66666---|----------------|-------88-----88|
A|----------------|----------------|66--------------|----------------|
E|444-------------|----------------|----------------|----------------|

e|----------------|----------------|----------------|----------------|
B|----------------|----------------|---------------6|---8------------|
G|--

In [35]:
tabs = display_tab.copy()
print_tabs(tabs)

e|----------------|----------------|----------------|----------------|
B|-----------88---|-------8888-----|---------------8|8---------------|
G|-----------7773-|----------------|----------------|-------7--------|
D|--------------88|888--------88---|---------8----5-|-----------88-6-|
A|----------------|----6-----------|---------------6|6---------------|
E|----------------|----------------|----------------|----------------|

e|----------------|----------------|----------------|----------------|
B|-------66------6|----------------|---------------8|----------------|
G|----------------|----------------|----------------|----------------|
D|6666--66------66|--------66666---|----------------|-------88-----88|
A|----------------|----------------|66--------------|----------------|
E|444-------------|----------------|----------------|----------------|

e|----------------|----------------|----------------|----------------|
B|----------------|----------------|---------------6|---8------------|
G|--

In [36]:
def print_tabs(tabs):
  """
  Takes a tabs dataframe and prints it as a formatted tablature
  """
  tab_list = ['e','B','G','D','A','E']
  len_long_row = len(str_row(tabs.loc[['e']]))
  num_div = 4                                  # Number of divisions per line
  num_lines = int(len_long_row/17//num_div+1)  # Number of lines for the tab
  for line in range(0,num_lines):              # Iterates over lines
    for index in tab_list:                     # Iterates over each index
      row = tabs.loc[[f'{index}']]             
      long_row = str_row(row)
      print(f"{index}|{long_row[num_div*17*line:num_div*17*(line+1)]}")
    print()

In [37]:
def web_tabs(tabs):
  """
  Takes a tabs dataframe and returns a multilinear string to feed the app
  """
  tab_list = ['e','B','G','D','A','E']
  len_long_row = len(str_row(tabs.loc[['e']]))
  num_div = 4                                  # Number of divisions per line
  num_lines = int(len_long_row/17//num_div+1)  # Number of lines for the tab
  line_list = []
  for line in range(0,num_lines):              # Iterates over lines
    for index in tab_list:                     # Iterates over each index
      row = tabs.loc[[f'{index}']]             
      long_row = str_row(row)
      line_list.append(f"{index}|{long_row[num_div*17*line:num_div*17*(line+1)]}")
    line_list.append("\n")
  multi_str = '\n'.join([line for line in line_list]) # joins all lines into a multistring
  return multi_str

In [ ]:
bbbb

In [38]:
print(web_tabs(display_tab))

e|----------------|----------------|----------------|----------------|
B|-----------88---|-------8888-----|---------------8|8---------------|
G|-----------7773-|----------------|----------------|-------7--------|
D|--------------88|888--------88---|---------8----5-|-----------88-6-|
A|----------------|----6-----------|---------------6|6---------------|
E|----------------|----------------|----------------|----------------|


e|----------------|----------------|----------------|----------------|
B|-------66------6|----------------|---------------8|----------------|
G|----------------|----------------|----------------|----------------|
D|6666--66------66|--------66666---|----------------|-------88-----88|
A|----------------|----------------|66--------------|----------------|
E|444-------------|----------------|----------------|----------------|


e|----------------|----------------|----------------|----------------|
B|----------------|----------------|---------------6|---8------------|
G|

### Getting predictions with npz


In [39]:
data = np.load( # load the pre -predicted file
    '00_BN1-129-Eb_comp.npz',
    allow_pickle=True)
file_list = data.files # files in data
print(file_list)

['repr', 'labels']


In [40]:
data['repr'].shape

(962, 192)

In [41]:
x_new_npz = repr_to_xnew(data["repr"])
y_pred_npz = model.predict(x_new_npz)
y_pred_npz.shape

(962, 6, 21)

In [42]:
#print y_pred only
labels_npz = y_pred_npz

#test it out
expanded_tab = make_tab(labels_npz,len(labels_npz))
display_tab = make_squeezed_tab(expanded_tab,5)
print_tabs(display_tab)

e|----------------|----------------|----------------|----------------|
B|-----------88---|-------8888-----|---------------8|8---------------|
G|-----------7773-|----------------|----------------|-------7--------|
D|--------------88|888--------88---|---------8----5-|-----------88-6-|
A|----------------|----6-----------|---------------6|6---------------|
E|----------------|----------------|----------------|----------------|

e|----------------|----------------|----------------|----------------|
B|-------66------6|----------------|---------------8|----------------|
G|----------------|----------------|----------------|----------------|
D|6666--66------66|--------66666---|----------------|-------88-----88|
A|----------------|----------------|66--------------|----------------|
E|444-------------|----------------|----------------|----------------|

e|----------------|----------------|----------------|----------------|
B|----------------|----------------|---------------6|---8------------|
G|--

In [43]:
display_tab

,0,5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95,100,105,110,115,120,125,130,135,140,145,150,155,160,165,170,175,180,185,190,195,...,765,770,775,780,785,790,795,800,805,810,815,820,825,830,835,840,845,850,855,860,865,870,875,880,885,890,895,900,905,910,915,920,925,930,935,940,945,950,955,960
E,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,4,4,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
A,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,6,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
D,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,8,8,8,8,8,-1,-1,-1,-1,-1,-1,-1,-1,8,8,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,6,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
G,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,7,7,7,3,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
B,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,8,8,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,8,8,8,8,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,6,-1,-1,-1,8,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
e,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [44]:
test_string = """
e|----------------|----------------|----------------|----------------|----------------|
B|-----------88---|-------8888-----|---------------8|8---------------|-------66------6|
G|-----------7773-|----------------|----------------|-------7--------|----------------|
D|--------------88|888--------88---|---------8----5-|-----------88-6-|6666--66------66|
A|----------------|----6-----------|---------------6|6---------------|----------------|
E|----------------|----------------|----------------|----------------|444-------------|\n

e|----------------|----------------|----------------|----------------|----------------|
B|----------------|---------------8|----------------|----------------|----------------|
G|----------------|----------------|----------------|----------------|---5------------|
D|--------66666---|----------------|-------88-----88|----------3---88|---------------6|
A|----------------|66--------------|----------------|----------------|----------------|
E|----------------|----------------|----------------|----------------|---------44-----|\n

e|----------------|----------------|-
B|---------------6|---8------------|-
G|----------------|----------------|-
D|----------------|----------------|-
A|----------------|----------------|-
E|----------------|----------------|-\n
"""

#st.write(test_string)

In [45]:
print(test_string)


e|----------------|----------------|----------------|----------------|----------------|
B|-----------88---|-------8888-----|---------------8|8---------------|-------66------6|
G|-----------7773-|----------------|----------------|-------7--------|----------------|
D|--------------88|888--------88---|---------8----5-|-----------88-6-|6666--66------66|
A|----------------|----6-----------|---------------6|6---------------|----------------|
E|----------------|----------------|----------------|----------------|444-------------|


e|----------------|----------------|----------------|----------------|----------------|
B|----------------|---------------8|----------------|----------------|----------------|
G|----------------|----------------|----------------|----------------|---5------------|
D|--------66666---|----------------|-------88-----88|----------3---88|---------------6|
A|----------------|66--------------|----------------|----------------|----------------|
E|----------------|----------

In [46]:
tablature = make_empty_tab()
for frame_idx in range(0, 100):
    frame = pd.DataFrame([0,0,0,0,0])
    tablature[frame_idx] = frame
tablature

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
E,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
G,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Comparing models

In [47]:
#Building model and loading weights
model2 = build_model()
model2.load_weights('weights_02_loss_6.2780_acc_0.7276.h5')
model2

In [48]:
#loading same x_new
x_new_test = repr_to_xnew(process)
x_new_test.shape

(962, 192, 9, 1)

In [49]:
# Getting predictions for both models
y_pred_0 = model.predict(x_new_test)
y_pred_2 = model2.predict(x_new_test)

In [50]:
x_new_n = np.array(x_new_test,dtype='float32')
y_pred_n=model.predict(x_new_n)
expanded_tab_0 = make_tab(y_pred_n,len(y_pred_n))
display_tab_0 = make_squeezed_tab(expanded_tab_0,5)
print_tabs(display_tab_0)


e|----------------|----------------|----------------|----------------|
B|-----------88---|-------8888-----|---------------8|8---------------|
G|-----------7773-|----------------|----------------|-------7--------|
D|--------------88|888--------88---|---------8----5-|-----------88-6-|
A|----------------|----6-----------|---------------6|6---------------|
E|----------------|----------------|----------------|----------------|

e|----------------|----------------|----------------|----------------|
B|-------66------6|----------------|---------------8|----------------|
G|----------------|----------------|----------------|----------------|
D|6666--66------66|--------66666---|----------------|-------88-----88|
A|----------------|----------------|66--------------|----------------|
E|444-------------|----------------|----------------|----------------|

e|----------------|----------------|----------------|----------------|
B|----------------|----------------|---------------6|---8------------|
G|--

In [51]:
expanded_tab_0 = make_tab(y_pred_0,len(y_pred_0))
display_tab_0 = make_squeezed_tab(expanded_tab_0,5)
print_tabs(display_tab_0)

e|----------------|----------------|----------------|----------------|
B|-----------88---|-------8888-----|---------------8|8---------------|
G|-----------7773-|----------------|----------------|-------7--------|
D|--------------88|888--------88---|---------8----5-|-----------88-6-|
A|----------------|----6-----------|---------------6|6---------------|
E|----------------|----------------|----------------|----------------|

e|----------------|----------------|----------------|----------------|
B|-------66------6|----------------|---------------8|----------------|
G|----------------|----------------|----------------|----------------|
D|6666--66------66|--------66666---|----------------|-------88-----88|
A|----------------|----------------|66--------------|----------------|
E|444-------------|----------------|----------------|----------------|

e|----------------|----------------|----------------|----------------|
B|----------------|----------------|---------------6|---8------------|
G|--

In [52]:
expanded_tab_2 = make_tab(y_pred_2,len(y_pred_2))
display_tab_2 = make_squeezed_tab(expanded_tab_2,5)
print_tabs(display_tab_2)

e|----------------|----------------|----------------|----------------|
B|----------8888--|-------88888----|---------------8|88--------------|
G|-----------7773-|----------------|---------------7|-------77-------|
D|--------------88|8888-------88---|---------8----8-|-----------88---|
A|----------------|----------------|--------------66|66--------------|
E|----------------|----------------|----------------|----------------|

e|----------------|----------------|----------------|----------------|
B|-------666-----6|----------------|-------8------88|---888-------8--|
G|----------------|----------------|----------------|-------------7--|
D|-------6------66|6--8----666-6---|----------------|-------88----888|
A|----------------|----------------|66--------------|----------------|
E|444-------------|----------------|----------------|----------------|

e|----------------|----------------|----------------|----------------|
B|----------------|---6------------|-8-------------6|---8------------|
G|--

## Experiment.wav

In [53]:
filenames="experiment.wav"
genrep = TabDataReprGen()
data_repr = genrep.load_rep_from_raw_file(filenames)
x_new_exp = repr_to_xnew(data_repr)
y_pred_0_exp = model.predict(x_new_exp)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:51: WavFileWarning: Chunk (non-data) not understood, skipping it.


ValueError: ignored

In [ ]:
filenames="00_Funk3-112-C#_solo_mic.wav"
genrep = TabDataReprGen()
data = genrep.load_rep_and_labels_from_raw_file(filenames)
data_repr_exp = np.swapaxes(genrep.preprocess_audio(data),0,1)
x_new_exp = repr_to_xnew(process)
y_pred_0_exp = model.predict(x_new_exp)

In [ ]:
wavfile.read("experimentmono.wav")

In [ ]:
#!pip install wavfile

In [ ]:

wavfile.read("00_Funk3-112-C#_solo_mic.wav")

In [ ]:
wavfile.read("test_mono.wav")